# The Hydrogen Wavefunction & Electron Density Cloud Plot
### Execution of File
Execution is done through the Python terminal by entering your quantum numbers with set restrictions stated below as well,

<div align="center">
  
| Quantum Number | Limitations                                 |
| -------------- | ------------------------------------------- |
| $$n$$          | $$n \geq  1$$                               |
| $$ℓ$$          | $$0 \leq  l \lt  n$$                        |
| $$m_ℓ$$        | $$-ℓ \leq m_ℓ \leq ℓ$$                      |

</div>

After execution, a 3D interactable figure will pop up via Matplotlib and will save a .png file within the respective Python file directory where ever that may be on your device. If the Wavefunction Plot is not within the bounds, try adjusting the Global Constants to a larger value by an addition of 500 for both. This portion is not automated, but will be in the future.

Figure examples are shown below for: $n = 2, l = 1, m_ℓ = 1$

<div align="center">
  <h3> Real Hydrogen Wavefunction </h3>
  <img src="Example_211_Real.png" />
  
  <h3> Complex Hydrogen Wavefunction </h3>
  <img src="Example_211_Complex.png" />

  <h3> Probability Density of Hydrogen Wavefunction </h3>
  <img src="Example_211_Probability_Density.png" />
</div>

For the different Python libraries, they are stated below along with the following exact import lines,

* Math (Internal)
* NumPY (1.24.3)
* SciPY (1.11.2)
* Matplotlib (3.7.2)

In [1]:
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

Following the imports, we will go through the different functions as they are portions of the total wavefunction equation.

$$\psi_{(n,ℓ,m_ℓ)} = R_{n,ℓ}(r)\times Y_{ℓ,m_ℓ}(\theta,\phi)$$

By setting a linear space for $(x,y,z)$ later on along with a meshgrid for our plot, we can start assuming these terms already exist by having $(r, \theta, \phi)$ as numpy arrays in the future. For a consistent plot, $np.linspace$ will require the extent of the plot and the number of points which we will declare as adjustable constants.

In [ ]:
n =  int(input('Enter an n Quantum Number: '))
l = int(input('Enter an l Quantum Number: '))
ml = int(input('Enter an ml Quantum Number: '))

#GLOBAL CONSTANTS (ADJUST IF NECESSARY)
num_points = 680
extent = 480


The Radial Contribution function is set as $R_{n,ℓ}(r)$ within the equation which is equivalent to,

$$R_{(n,ℓ)}(r) = \sqrt{\left(\frac{2}{na_0}\right)^{3}\frac{(n-ℓ-1)!}{2n(n+1)!}}\left(\frac{2r}{na_0}\right)^{ℓ} L_{n-ℓ-1}^{2ℓ+1}\left(\frac{2r}{na_0}\right)e^{-\frac{r}{na_0}}$$


The following function within the Python script is,

In [2]:
# Computes the Radial Contribution of the Wavefunction Using Exponential Decay, Laguerre Polynomials, and Power Term
#
# Arguments: (int) n - Principal Quantum Number, the energy level or shell. (r in spherical coordinates)
#            (int) l - Angular Momentum Quantum Number, the subshell or angular dependence of the orbital. (Θ in spherical coordinates)
#            (numpy array) r - Variable Radius Coordinate for plot
#            
# Output: (numpy array) of the radial contribution to the wavefunction
def radial_Contribution(n, l, r):
    
    e = np.e
    a0 = sp.constants.physical_constants["Bohr radius"][0] * 1e+12
    z = 2 / (n * a0)

    squareRootTerm = np.sqrt(
                            (z ** 3) *
                            ((np.math.factorial(n - l - 1)) / (2 * n * (np.math.factorial(n + l))))
                            )
    
    powerOfLTerm = (z * r) ** l

    laguerreTerm = sp.special.genlaguerre((n - l - 1), (2 * l + 1))

    decayTerm = e**((-1)*((z * r) / 2))

    return squareRootTerm * powerOfLTerm * laguerreTerm(z * r) * decayTerm

This will return the Radial Contribution of the wavefunction for each $(r)$ value.


The Angular Contribution function is set as $Y_{ℓ}^{m_ℓ}(θ,ϕ)$ within the equation which is equivalent to,

$$Y_{ℓ}^{m_ℓ}(θ,ϕ) = (-1)^{m_ℓ} \sqrt{\left(\frac{2ℓ+1}{4π}\right)\frac{(ℓ-m_ℓ)!}{(ℓ+m_ℓ)!}}P_{ℓ}^{m_ℓ}(cos(θ))e^{im_ℓϕ}$$


The following function within the Python script is,

In [ ]:
# Computes the Angular Contribution of the Wavefunction Using Exponential Decay, Legendre Polynomials, and Power Term
#
# Arguments: (int) l - Angular Momentum Quantum Number, the subshell or angular dependence of the orbital. (Θ in spherical coordinates)
#            (int) ml - Magnetic Quantum Number, the orientation in space of an orbital (ϕ in spherical coordinates)
#            (numpy array) theta - Variable Polar Angle Coordinate for plot
#            (numpy array) phi - Variable Azimuth Angle Coordinate for plot
#           
# Output: (numpy array) of the angular contribution (spherical harmonics) of the wavefunction
def angular_Contribution(l, ml, theta, phi):

    e = np.e
    pi = np.pi

    negPosTerm = (-1)**ml

    #Needed to ensure calculation for -l
    absML = np.abs(ml)
    sqaureRootTerm = np.sqrt(
                            (((2 * l) + 1)/(4 * pi)) *
                            ((np.math.factorial(l - absML))/(np.math.factorial(l + absML)))
                            )
    
    legendreTerm = sp.special.lpmv(ml, l, np.cos(theta))

    decayTerm = np.exp(1.j * ml * phi)

    return negPosTerm * sqaureRootTerm * legendreTerm * decayTerm

This will return the Radial Contribution of the wavefunction for each $(\theta, \phi)$ value being of complex nature.

Computing the Wavefunction by following, 
$$\psi_{(n,ℓ,m_ℓ)} = R_{n,ℓ}(r)\times Y_{ℓ,m_ℓ}(\theta,\phi)$$

requires error checking for the quantum numbers along with setting our $np.linspace$ as previously mentioned for $(x,y,z)$ and converting them to their respective spherical coordinate by the following translations, 
$$r,ρ = \sqrt{x^2 + y^2 + z^2}$$
$$\theta = \arctan{\frac{y}{x}}$$
$$\phi = \arccos{\frac{z}{\sqrt{x^2 + y^2 + z^2}}}$$

The resulting function is as follows,

In [ ]:
# Computes the total complex wavefunction from the calculated angular_Contribution and radial_Contribution
#
# Arguments: (int) n - Principal Quantum Number, the energy level or shell. (r in spherical coordinates)
#            (int) l - Angular Momentum Quantum Number, the subshell or angular dependence of the orbital. (Θ in spherical coordinates) 
#            (int) ml - Magnetic Quantum Number, the orientation in space of an orbital (ϕ in spherical coordinates)
#            **IMPORTANT, with converting r, theta, phi to their respective cartesian coordinates. We must consider all possible values of r, Θ, ϕ.
#            *Note, x,y,z linspaces are set from the global constants
#          
# Output: COMPLEX (numpy array) of the total wavefunction
def complex_Wave_Function(n, l, ml):     

    if (not isinstance(n, int)) or (n < 1):
        raise ValueError('Quantum Number Error: n shall be part of the Natural Set (n >= 1)')

    if (not isinstance(l, int)) or not (0 <= l < n):
        raise ValueError('Quantum Number Error: l shall be part of the Positive Integer Set and Satisfy (0 <= l < n)')
    
    if (not isinstance(ml, int)) or not (-l <= ml <= l):
        raise ValueError('Quantum Number Error: ml shall be part of the Integer Set and Satisfy (-l <= ml <= l)')

    pi = np.pi

    x = y = z = np.linspace(-extent, extent, num_points)
    x, y = np.meshgrid(x, y)

    magnitude = np.sqrt((x ** 2) + (y ** 2) + (z ** 2))
    r = magnitude
    theta = np.arctan(y / x)
    phi = np.arccos(z / magnitude)

    radial = radial_Contribution(n, l, r)
    angular = angular_Contribution(l, ml, theta, phi)

    psi = radial * angular
    
    return psi

This will return the total complex wavefunction for each $(r, \theta, \phi)$.

Determining the probability density function $P(r,\theta,\phi)$ is followed by a seperate simple equation as stated below, 

$$P(r,\theta,\phi) = |\psi_{(n,ℓ,m_ℓ)}(r,\theta,\phi)|^2$$

which follows a fairly simple function return stated as, 

In [ ]:
# Computes the probability density of any given wavefunction set as z for future plot
#
# Arguments: (numpy array) psi - A COMPLEX numpy array describing the total wavefunction  
# 
# Output: REAL (numpy array) of the probability density
def probability_Density(psi):

    return (np.abs(psi) ** 2)

Now following the main plot of any given function was to interpret the function value ($(\psi)$ in this case) as our main $(z)$ points for a consistent set of $(x)$ and $(y)$.

Along with using an external input of the plot title for pretty printing, we can save these plots as different file names and give them different titles by a neat trick!

In [ ]:
# Plots any given REAL function including probability density P on a 3D contour graph
#
# Arguments: (numpy array) P - A REAL numpy array describing the total function  
#            (string) text - A title formatter for Real, Complex, or Probability Density Plot
# 
# Output: A final plot of the given function
def main_Plot_Wavefunction(P, text):

    x = y = np.linspace(-extent, extent, num_points)
    x, y = np.meshgrid(x, y)
    z = P

    fig = plt.figure()
    ax = plt.axes(projection = '3d')
    plt.title("Hydrogen Wavefunction " + text + "$ \psi_{(n,ℓ,m_ℓ)}$")
    ax.set_xlabel('$x$', fontsize=20, rotation=150)
    ax.set_ylabel('$y$', fontsize=20, rotation=0)
    ax.set_zlabel('$\psi_{(n,ℓ,m_ℓ)}$', fontsize=20, rotation=150)
    
    ax.contour(x, y, z, 50, cmap='magma')

    plt.savefig(f'({n},{l},{ml})[{text}].png')
    plt.show()

Now with all our functions set, we use the following function calls stated as, 

In [ ]:
#Real Plot
main_Plot_Wavefunction(np.real(complex_Wave_Function(n, l, ml)), 'Real')

#Complex Plot
main_Plot_Wavefunction(np.imag(complex_Wave_Function(n, l, ml)), 'Complex')

#Probability Density Plot
main_Plot_Wavefunction(probability_Density(complex_Wave_Function(n, l, ml)), 'Probability Density')

and input our Quantum Number Values following the restrictions and boundary conditions resulting our Matplotlib 3D contour plot and some new saved figures.